In [4]:
import pandas as pd
import os

os.chdir('/Users/johnngai/Desktop/final_project/ATP_matches_past_10_years_record/')

In [194]:
# Read the cleaned csv
data=pd.read_csv('grand_slam_database.csv')

In [195]:
# Drop 'Unnamed: 0'
data.drop(columns=['Unnamed: 0'],inplace=True)

In [196]:
# Locate the tournament info columns
tourney=data.iloc[:,:5]

In [197]:
# Split the tournament info from main dataframe

tournament=tourney['tourney_name'].unique()

In [198]:
tournament

array(['Australian Open', 'Roland Garros', 'Wimbledon', 'US Open'],
      dtype=object)

In [199]:
# Locate the matches info columns
match=data.iloc[:,5:]

In [200]:
# Create function to assign tournament name
def assign_tournament(x):
    if x==tournament[0]:
        x='A'
    elif x==tournament[1]:
        x='R'
    elif x==tournament[2]:
        x='W'
    elif x==tournament[3]:
        x='U'
    return x

In [201]:
# Apply it to the main dataframe
data['id_1']=data['tourney_name'].apply(assign_tournament)

In [202]:
# Create function to assign tournament year
def assign_year(x):
    return x.split('-')[0]

In [203]:
# Apply it to the main dataframe
data['id_2']=data['tourney_id'].apply(assign_year)

In [204]:
# Create a column combining tournament name and tournament year and to be used as the key for primary/foreign keys in Google Cloud database
id=pd.Series(data['id_1']+'_'+data['id_2'])

In [210]:
# Create tournament table for database
tourney_table=pd.concat([id,tourney],axis=1).rename(columns={0:'id'})

In [211]:
tourney_table=tourney_table.groupby(['id','tourney_name','surface','tourney_level','tourney_date']).count().reset_index().drop(['tourney_id'],axis=1)

In [212]:
tourney_table.head()

,id,tourney_name,surface,tourney_level,tourney_date
0,A_2008,Australian Open,Hard,G,2008-01-14
1,A_2009,Australian Open,Hard,G,2009-01-19
2,A_2010,Australian Open,Hard,G,2010-01-18
3,A_2011,Australian Open,Hard,G,2011-01-17
4,A_2012,Australian Open,Hard,G,2012-01-16


In [214]:
# Create match table for database
match_table=pd.concat([id,match],axis=1).rename(columns={0:'id'})

In [228]:
match_table.head()

,id,winner_name,winner_hand,winner_ht,winner_ioc,winner_rank,loser_name,loser_hand,loser_ht,loser_ioc,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,1st_set,2nd_set,3rd_set,4th_set,5th_set
0,A_2018,Rafael Nadal,L,185,ESP,1,Victor Estrella,R,173,DOM,...,19,5,10,4,12,6-1,6-1,6-1,NaN,NaN
1,A_2018,Damir Dzumhur,R,172,BIH,30,Paolo Lorenzi,R,183,ITA,...,68,41,24,8,13,3-6,2-6,7-6(5),6-2,6-4
2,A_2018,Diego Sebastian Schwartzman,R,170,ARG,26,Dusan Lajovic,R,180,SRB,...,78,24,29,13,24,2-6,6-3,5-7,6-4,11-9
3,A_2018,Alexandr Dolgopolov,R,180,UKR,37,Andreas Haider Maurer,R,190,AUT,...,40,17,16,9,16,7-6(3),6-3,6-4,NaN,NaN
4,A_2018,Matthew Ebden,R,188,AUS,78,John Isner,R,206,USA,...,48,20,19,0,4,6-4,3-6,6-3,6-3,NaN


In [ ]:
import pymysql
from sqlalchemy import create_engine


driver='mysql+pymysql:'
user='root'
pw='10116'
ip='34.65.159.156'
db='grandslam'

connection_string=f'{driver}//{user}:{pw}@{ip}/{db}'

engine=create_engine(connection_string)

In [227]:
# Export match table to database
match_table.to_sql('matches', engine, if_exists = 'append', index=False)

In [ ]:
# Export tournament table to database
tourney_table.to_sql('tournament', engine, if_exists = 'append', index=False)